In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats.distributions as dist
import matplotlib as plt

In [17]:
da = pd.read_csv("nhanes_2015_2016.csv")

da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [5]:
#Hypothesis Tests for One Proportion

#Is the % of lifetime smokers in the US != 40%

x = da.SMQ020x.dropna() == "Yes"
p = x.mean()
se = np.sqrt(.4 * (1 - .4)/ len(x))
test_stat = (p - 0.4) / se

pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pvalue)

0.7823563854332805 0.4340051581348052


In [13]:
#Same thing but with stats models

#Normal aprox with estimated proportion in SE
print(sm.stats.proportions_ztest(x.sum(), len(x), 0.4))
#Normal aprox with null proportion in SE
print(sm.stats.proportions_ztest(x.sum(), len(x), 0.4, prop_var = 0.4))
#Exact binomial p-value
print(sm.stats.binom_test(x.sum(), len(x), 0.4))

(0.7807518954896244, 0.43494843171868214)
(0.7823563854332805, 0.4340051581348052)
0.4340360854410036


C:\Users\kevin\anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:67: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


In [20]:
#Hypothesis Tests for Two Proportions

#Are lifetime smoking rates between of US females and males age 20-25 significantly different?

#Drop missing values
dx = da[["SMQ020x", "RIDAGEYR", "RIAGENDRx"]].dropna()
#restrict age to 20-25
dx = dx.loc[(dx.RIDAGEYR >= 20) & (dx.RIDAGEYR <= 25), :]

#Summarize data by calculating proprtion of yeses and sample size
p = dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z == "Yes"), "size"])
p.columns = ["Smoke", "N"]
print(p)

print('\n')

#Pooled rate of yeses and SE of estimated difference of proportions
p_comb = (dx.SMQ020x == "Yes").mean()
va = p_comb * (1 - p_comb)
se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))

#Test statistic and p-value
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, p_value)

              Smoke    N
RIAGENDRx               
Female     0.238971  272
Male       0.341270  252


-2.5833303066279414 0.009785159057508375


In [21]:
#Same thing but with stats models
dx_females = dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_males = dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"].replace({"Yes": 1, "No": 0})

sm.stats.ttest_ind(dx_females, dx_males)

(-2.5949731446269344, 0.00972590232121254, 522.0)

In [33]:
#Is the mean of the systollic blood pressure for males age 40-50 !=120?

#Isolate data
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDRx']].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 40) & (dx.RIDAGEYR <= 50)  & (dx.RIAGENDRx == 'Male'), :]

#Mean of the systollic blood pressure
print(dx.BPXSY1.mean())

#Test statistic and p-value
sm.stats.ztest(dx.BPXSY1, value = 120)

125.86698337292161


(7.469764137102597, 8.033869113167905e-14)

In [38]:
#Compare the mean of males and females age 50-60, is there a significant difference?

#Isolate data
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDRx']].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
bpx_female = dx.loc[dx.RIAGENDRx == 'Female', 'BPXSY1']
bpx_male = dx.loc[dx.RIAGENDRx == 'Male', 'BPXSY1']

#Means
print(bpx_female.mean(), bpx_male.mean())
#Test statistic and p-value
print(sm.stats.ztest(bpx_female, bpx_male))
##Test statistic, p-value and degrees of freedom
print(sm.stats.ttest_ind(bpx_female, bpx_male))



127.92561983471074 129.23829787234044
(-1.105435895556249, 0.2689707570859362)
(-1.105435895556249, 0.26925004137768577, 952.0)


In [39]:
#Heteroscedacity of the data on BMI

dx = da[['BMXBMI', 'RIDAGEYR', 'RIAGENDRx']].dropna()
da['agegrp'] = pd.cut(dx.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
da.groupby(['agegrp' ,'RIAGENDRx'])['BMXBMI'].agg(np.std).unstack()

RIAGENDRx,Female,Male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


In [44]:
#Pooled and unequal aproach of variance estimation, difference between them show level of 
#variance and therefore heteroscedacity

for k, v in da.groupby('agegrp'):
    bmi_female = v.loc[v.RIAGENDRx == 'Female', 'BMXBMI'].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDRx == 'Male', 'BMXBMI'].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("Pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar = 'pooled'))
    print("Equal: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar = 'pooled'))
    print()

(18, 30]
Pooled:  (1.702693293364314, 0.08862548061450115)
Equal:  (1.702693293364314, 0.08862548061450115)

(30, 40]
Pooled:  (1.4378280405644988, 0.15048285114647975)
Equal:  (1.4378280405644988, 0.15048285114647975)

(40, 50]
Pooled:  (2.8933761158070186, 0.003811246059501354)
Equal:  (2.8933761158070186, 0.003811246059501354)

(50, 60]
Pooled:  (3.3621087799813747, 0.0007734964571391533)
Equal:  (3.3621087799813747, 0.0007734964571391533)

(60, 70]
Pooled:  (3.6172401442432602, 0.0002977610210319532)
Equal:  (3.6172401442432602, 0.0002977610210319532)

(70, 80]
Pooled:  (2.926729252512241, 0.003425469414486057)
Equal:  (2.926729252512241, 0.003425469414486057)

